# Talk like the President

In this notebook I build up towards a simple char RNN architecture using the [fastai library](https://github.com/fastai/).

The code is taken nearly line for line from [Lesson 6](https://github.com/fastai/fastai/blob/master/courses/dl1/lesson6-rnn.ipynb) of the FastAI Part 1 v2 course. The diagrams come from the course videos.

Jeremy provides a great overview of each of the architectures in the [lecture 6 video](https://youtu.be/sHcLkfRrgoQ?t=1h12m40s). 

In [1]:
import glob
from utils import *

In [2]:
from fastai.io import *
from fastai.conv_learner import *

from fastai.column_data import *

In [3]:
# create data dir if doesn't exist
!mkdir -p data

# Download and unzip corpus of speeches
!wget 'http://www.thegrammarlab.com/?wpdmpro=corpus-of-presidential-speeches&wpdmdl=595' -O "data/corpus.zip"
!unzip data/corpus.zip -d data/

# Data from:
#   Brown, D. W. (2016) Corpus of Presidential Speeches. Retrieved from http://www.thegrammarlab.com

--2018-01-26 15:22:26--  http://www.thegrammarlab.com/?wpdmpro=corpus-of-presidential-speeches&wpdmdl=595
Resolving www.thegrammarlab.com (www.thegrammarlab.com)... 192.124.249.2
Connecting to www.thegrammarlab.com (www.thegrammarlab.com)|192.124.249.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8402362 (8.0M) [application/zip]
Saving to: ‘data/corpus.zip’

data/corpus.zip     100%[===================>]   8.01M   800KB/s    in 10s     

2018-01-26 15:22:41 (785 KB/s) - ‘data/corpus.zip’ saved [8402362/8402362]

Archive:  data/corpus.zip
   creating: data/Corpus of Presential Speeches/adams/
  inflating: data/Corpus of Presential Speeches/adams/adams_speeches_000.txt  
  inflating: data/Corpus of Presential Speeches/adams/adams_speeches_001.txt  
  inflating: data/Corpus of Presential Speeches/adams/adams_speeches_002.txt  
  inflating: data/Corpus of Presential Speeches/adams/adams_speeches_003.txt  
  inflating: data/Corpus of Presential Speeches/adams/ada

  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_001.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_002.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_003.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_004.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_005.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_006.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_007.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_008.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_009.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_010.txt  
  inflating: data/Corpus of Presential Speeches/coolidge/coolidge_speeches_011.txt  
   creating: data/Corpus of Presential Speeches/eisenhower/
  inf

  inflating: data/Corpus of Presential Speeches/jefferson/jefferson_speeches_017.txt  
  inflating: data/Corpus of Presential Speeches/jefferson/jefferson_speeches_018.txt  
  inflating: data/Corpus of Presential Speeches/jefferson/jefferson_speeches_019.txt  
  inflating: data/Corpus of Presential Speeches/jefferson/jefferson_speeches_020.txt  
  inflating: data/Corpus of Presential Speeches/jefferson/jefferson_speeches_021.txt  
  inflating: data/Corpus of Presential Speeches/jefferson/jefferson_speeches_022.txt  
  inflating: data/Corpus of Presential Speeches/jefferson/jefferson_speeches_023.txt  
   creating: data/Corpus of Presential Speeches/johnson/
  inflating: data/Corpus of Presential Speeches/johnson/johnson_speeches_000.txt  
  inflating: data/Corpus of Presential Speeches/johnson/johnson_speeches_001.txt  
  inflating: data/Corpus of Presential Speeches/johnson/johnson_speeches_002.txt  
  inflating: data/Corpus of Presential Speeches/johnson/johnson_speeches_003.txt  
  

  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_009.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_010.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_011.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_012.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_013.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_014.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_015.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_016.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_017.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_018.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_019.txt  
  inflating: data/Corpus of Presential Speeches/obama/obama_speeches_020.txt  
  inflating: data/Corpus of Presential Speeches/obam

  inflating: data/Corpus of Presential Speeches/tyler/tyler_speeches_011.txt  
  inflating: data/Corpus of Presential Speeches/tyler/tyler_speeches_012.txt  
  inflating: data/Corpus of Presential Speeches/tyler/tyler_speeches_013.txt  
  inflating: data/Corpus of Presential Speeches/tyler/tyler_speeches_014.txt  
  inflating: data/Corpus of Presential Speeches/tyler/tyler_speeches_015.txt  
  inflating: data/Corpus of Presential Speeches/tyler/tyler_speeches_016.txt  
  inflating: data/Corpus of Presential Speeches/tyler/tyler_speeches_017.txt  
   creating: data/Corpus of Presential Speeches/vanburen/
  inflating: data/Corpus of Presential Speeches/vanburen/vanburen_speeches_000.txt  
  inflating: data/Corpus of Presential Speeches/vanburen/vanburen_speeches_001.txt  
  inflating: data/Corpus of Presential Speeches/vanburen/vanburen_speeches_002.txt  
  inflating: data/Corpus of Presential Speeches/vanburen/vanburen_speeches_003.txt  
  inflating: data/Corpus of Presential Speeches/v

In [4]:
# Data taken from:
#   Brown, D. W. (2016) Corpus of Presidential Speeches. Retrieved from http://www.thegrammarlab.com
get_data('http://www.thegrammarlab.com/?wpdmpro=corpus-of-presidential-speeches&wpdmdl=595', 'data')

In [5]:
speeches = ' '.join(preprocess_speech(file) for file in glob('data/Corpus of Presential Speeches/obama/*'))

In [6]:
len(speeches)

1027441

Just over 1 mln chars! This should be perfect for our purposes!

Let's start with creating a vocabulary of all the chars that are used.

In [7]:
chars = sorted(list(set(speeches)))
vocab_size = len(chars)
print('total chars:', vocab_size)

total chars: 74


In [8]:
''.join(chars)

' !"$\',-./0123456789:;>?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz'

We want to be able to easily go from char to its index in the vocabulary and back. Let's use dictionaries for that.

In [9]:
char2idx = {c: idx for idx, c in enumerate(chars)}
idx2char = {idx: c for idx, c in enumerate(chars)}

Now we need to convert the speeches to representation as list of corresponding indexes.

In [10]:
idxs = [char2idx[char] for char in speeches]

In [11]:
idxs[:11]

[35, 65, 7, 0, 41, 63, 52, 48, 58, 52, 65]

In [12]:
''.join(idx2char[idx] for idx in idxs[:11])

'Mr. Speaker'

## First model - simple FC NN

Our first model will be very simple - we will use three chars to predict the fourth char. In order to do so, we embed each char using a rank 1 tensor of length 42. We concatenate the embeddings together and we slap a fully connected layer on top.

It is a simple architecture that I wanted to try before we move onto RNNs!

<img src='images/basic_nn.png'>

In [13]:
n_fac = 42 # number of latent factors, embedding length
n_hidden = 256

In [14]:
class FCModel(nn.Module):
    def __init__(self, vocab_size, n_fac, n_hidden):
        super().__init__()
        self.n_fac = n_fac
        self.e = nn.Embedding(vocab_size, n_fac)

        # fully connected layer
        self.l_in = nn.Linear(n_fac * 3, n_hidden)

        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        inp = torch.cat([self.e(c1), self.e(c2), self.e(c3)], dim=1)
        h = F.relu(self.l_in(inp))
        
        return F.log_softmax(self.l_out(h))

### Create inputs

In [15]:
cs=3
c1_dat = idxs[0:-cs]
c2_dat = idxs[1:-cs+1]
c3_dat = idxs[2:-cs+2]
c4_dat = idxs[3:]

Visually inspecting whether the above worked!

In [16]:
print(list(idx2char[idx] for idx in c1_dat[-10:]))
print(list(idx2char[idx] for idx in c2_dat[-10:]))
print(list(idx2char[idx] for idx in c3_dat[-10:]))
print(list(idx2char[idx] for idx in c4_dat[-10:]))

['a', '.', ' ', 'T', 'h', 'a', 'n', 'k', ' ', 'y']
['.', ' ', 'T', 'h', 'a', 'n', 'k', ' ', 'y', 'o']
[' ', 'T', 'h', 'a', 'n', 'k', ' ', 'y', 'o', 'u']
['T', 'h', 'a', 'n', 'k', ' ', 'y', 'o', 'u', '.']


In [17]:
x1 = np.stack(c1_dat)
x2 = np.stack(c2_dat)
x3 = np.stack(c3_dat)
y = np.stack(c4_dat)

In [18]:
x1.shape, x2.shape, x3.shape, y.shape

((1027438,), (1027438,), (1027438,), (1027438,))

Let's create our dataset from tese four arrays.

In [19]:
# This one line allows me to run my models on a subset of data while I work on this notebook
# md = ColumnarModelData.from_arrays('.', [-20_000], np.stack([x1,x2,x3], axis=1)[:200_000, :], y[:200_000], bs=2**9)

md = ColumnarModelData.from_arrays('.', [-100_000], np.stack([x1,x2,x3], axis=1), y, bs=2**9)

In [20]:
m = FCModel(vocab_size, n_fac, n_hidden).cuda()

In [21]:
opt = optim.Adam(m.parameters(), 1e-2)

fit(m, md, 1, opt, F.nll_loss)
set_lrs(opt, 1e-3)
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.61588  0.55123]                                   



HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.49669  0.67421]                                   



Let's see if our model learned anything useful!

In [22]:
def get_next(inp):
    idxs = T(np.array([char2idx[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.argmax(to_np(p))
    return idx2char[i]

In [23]:
get_next('pre')

's'

In [24]:
get_next(' an')

'd'

Let's try to produce something a bit longer.

In [25]:
text = 'Goo'
for i in range(200): text += get_next(text[-3:])

print(text)

Good the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stand the stan


That was bound to happen with such a simple model. Let's see if we can introduce some randomness here.

In [26]:
def get_next_stochastic(inp):
    idxs = T(np.array([char2idx[c] for c in inp]))
    p = m(*VV(idxs))
    i = np.random.choice(vocab_size, p=np.exp(p.data.cpu().numpy()[0]))
    return idx2char[i]

text = 'Goo'
for i in range(200): text += get_next_stochastic(text[-3:])

print(text)

Gook, and paging press, we security we they pol can depenstep a dism. Throughs, well, Press cause ofterms of has we respofits of help that we hard condents to here in pland to vote through local this a d


Well, this does sound like a president, but not the one whose manner of speech we are trying to emulate here. Let's try a slightly more complex architecture.

## Char3Model

Technically, I am not sure if this is an RNN. It looks to me like an unrolled RNN but maybe it is missing something that it needs to be called an unrolled RNN. Either way, this is just semantics and we are very close to having a 'vanilla' RNN.

<img src='images/3char.png'>

In [27]:
class Char3Model(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)

        # The 'green arrow' from our diagram - the layer operation from input to hidden
        # We are now feeding in chars one at a time! The input dimension of the first linear layer
        # is (1, 42) vs the (1, 3 * 42) we used earlier.
        self.l_in = nn.Linear(n_fac, n_hidden)

        # The 'orange arrow' from our diagram - the layer operation from hidden to hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        
        # The 'blue arrow' from our diagram - the layer operation from hidden to output
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, c1, c2, c3):
        in1 = F.relu(self.l_in(self.e(c1)))
        in2 = F.relu(self.l_in(self.e(c2)))
        in3 = F.relu(self.l_in(self.e(c3)))
        
        h = V(torch.zeros(in1.size()).cuda()) # initiating our hidden state to all zeros
        h = F.tanh(self.l_hidden(h+in1))
        h = F.tanh(self.l_hidden(h+in2))
        h = F.tanh(self.l_hidden(h+in3))
        
        return F.log_softmax(self.l_out(h))

In [28]:
m = Char3Model(vocab_size, n_fac).cuda()

In [29]:
opt = optim.Adam(m.parameters(), 1e-2)

fit(m, md, 1, opt, F.nll_loss)
set_lrs(opt, 1e-3)
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.83884  1.45692]                                   



HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.59302  1.98777]                                   



Let's run our two tests to see if it learned anything!

In [30]:
text = 'Goo'
for i in range(200): text += get_next(text[-3:])

print(text)

Good the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the th


In [31]:
text = 'Goo'
for i in range(200): text += get_next_stochastic(text[-3:])

print(text)

Googgestice's costs to brountrong-textried othe put cut be acativernessary or instor we doese to recop to may or not we in the from As we the bbest a whet from that to rege crealth 30 - look a is flority


Time to change our Char3Model into a 'real' RNN!

## CharLoopModel

<img src='images/simple_RNN.png'>

In [32]:
cs=8 # our model will generate predictions based on it seeing this many chars 

c_in_dat = [[idxs[i+j] for i in range(cs)] for j in range(len(idxs)-cs-1)]
c_out_dat = [idxs[j+cs] for j in range(len(idxs)-cs-1)]

xs = np.stack(c_in_dat, axis=0)
y = np.stack(c_out_dat)

In [33]:
xs.shape

(1027432, 8)

In [34]:
[[idx for idx in row] for row in xs[:8]]

[[35, 65, 7, 0, 41, 63, 52, 48],
 [65, 7, 0, 41, 63, 52, 48, 58],
 [7, 0, 41, 63, 52, 48, 58, 52],
 [0, 41, 63, 52, 48, 58, 52, 65],
 [41, 63, 52, 48, 58, 52, 65, 5],
 [63, 52, 48, 58, 52, 65, 5, 0],
 [52, 48, 58, 52, 65, 5, 0, 35],
 [48, 58, 52, 65, 5, 0, 35, 65]]

In [35]:
[[idx2char[idx] for idx in row] for row in xs[:cs]]

[['M', 'r', '.', ' ', 'S', 'p', 'e', 'a'],
 ['r', '.', ' ', 'S', 'p', 'e', 'a', 'k'],
 ['.', ' ', 'S', 'p', 'e', 'a', 'k', 'e'],
 [' ', 'S', 'p', 'e', 'a', 'k', 'e', 'r'],
 ['S', 'p', 'e', 'a', 'k', 'e', 'r', ','],
 ['p', 'e', 'a', 'k', 'e', 'r', ',', ' '],
 ['e', 'a', 'k', 'e', 'r', ',', ' ', 'M'],
 ['a', 'k', 'e', 'r', ',', ' ', 'M', 'r']]

In [36]:
class CharLoopModel(nn.Module):
    # This is an RNN!
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden)
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, n_hidden).cuda())
        for c in cs:
            inp = F.relu(self.l_in(self.e(c)))
            h = F.tanh(self.l_hidden(h+inp))
        
        return F.log_softmax(self.l_out(h))

In [37]:
# This one line allows me to run my models on a subset of data while I work on this notebook
# md = ColumnarModelData.from_arrays('.', [-20_000], xs[:200_000, :], y[:200_000], bs=2**9)

md = ColumnarModelData.from_arrays('.', [-100_000], xs, y, bs=2**9)

In [38]:
m = CharLoopModel(vocab_size, n_fac).cuda()

In [39]:
opt = optim.Adam(m.parameters(), 1e-2)

fit(m, md, 1, opt, F.nll_loss)
set_lrs(opt, 1e-3)
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.87007  2.43004]                                  



HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.56679  3.12984]                                  



In [40]:
get_next('Good mo')

'r'

In [41]:
text = 'Good mo'
for i in range(200): text += get_next(text[-8:])

print(text)

Good more the some the some the some the some the some the some the some the some the some the some the some the some the some the some the some the some the some the some the some the some the some the some


In [42]:
text = 'Good mo'
for i in range(200): text += get_next_stochastic(text[-8:])

print(text)

Good month. Them, welity work at it sel a people for there opportagen are the courstoryers not us I to science a where where in tway work good American the low for Must is were fooce of thnity to - Mind the 


Looking a bit better but still not that great. Next step - replacing the for loop with a PyTorch component.

## CharRnn

This model will do exactly the same thing as the model above, the difference being that it no longer relies on the manual for loop. We will replace it with a nn.RNN layer.

This is what that layer does (taken from [PyTorch documentation](http://pytorch.org/docs/master/nn.html#recurrent-layers)):

$$h_t = \tanh(w_{ih} * x_t + b_{ih}  +  w_{hh} * h_{(t-1)} + b_{hh})$$

In [43]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        self.e = nn.Embedding(vocab_size, n_fac)
#         self.l_in = nn.Linear(n_fac, n_hidden) <- nn.RNN will do this for us already so this can go
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        
    def forward(self, *cs):
        bs = cs[0].size(0)
        h0 = V(torch.zeros(1, bs, n_hidden).cuda())
        inp = self.e(torch.stack(cs))
        
        # output.shape = [8, bs, n_hidden]
        # hn.shape = [1, bs, n_hidden]
        #   turns out hn is the 'tensor containing the hidden state for k=seq_len' but for just a plain nn.RNN
        #   that ends up being output[-1, bs, n_hidden] which can be quite confusing!
        output, hn = self.rnn(inp, h0)
        return F.log_softmax(self.l_out(output[-1]))

In [44]:
m = CharRNN(vocab_size, n_fac).cuda()

In [45]:
opt = optim.Adam(m.parameters(), 1e-2)

fit(m, md, 1, opt, F.nll_loss)
set_lrs(opt, 1e-3)
fit(m, md, 1, opt, F.nll_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.79412  3.36834]                                  



HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

[ 0.       1.48315  4.21295]                                  



In [46]:
get_next('Good mo')

'r'

In [47]:
text = 'Good mo'
for i in range(200): text += get_next(text[-8:])

print(text)

Good more the program the program the program the program the program the program the program the program the program the program the program the program the program the program the program the program the p


In [48]:
text = 'Good mo'
for i in range(200): text += get_next_stochastic(text[-8:])

print(text)

Good mora. Build there its their it's to all ertam. And onle-placed. Communis of over with the cairs end doute a see promimo the tell-to payinst with that the familize,, about sirsh and who cublib, and offic


## What is going on here? 

I did some copying of code from Jeremy's notebook. Things seem to work. I am starting to use nn.RNN which hides some of the complexity (and I think I understand what it does).

But something feels a bit off. Would I be able to create a model from scratch?

At this point, all this seems quite fuzzy. The only way to really learn this is to build a model from scratch myself

## Adding a string of numbers

I will build a very simple RNN for evaluating a simple calculation contained in a string, for example '2+3'. For this example, the network should output the number 5.

I understand that this will be doing very little, but that is the plan. There are so many ways that this could be extended. Including other types of calculations, summing up digits in a string up until some special character indicating going back to zero (for testing forget gates), etc.

Let's see if I can get the most basic of models to work.

### Creating the dataset 

In [49]:
summands = [[x1, x2] for x1 in range(10) for x2 in range(10)]

y = map(lambda lst: lst[0] + lst[1], summands)
y = np.stack(y).reshape((-1,1))

xs = map(lambda lst: [str(lst[0]), '+', str(lst[1])], summands)
xs = np.stack(xs)

In [50]:
y[::16]

array([[ 0],
       [ 7],
       [ 5],
       [12],
       [10],
       [ 8],
       [15]])

In [51]:
xs[::16]

array([['0', '+', '0'],
       ['1', '+', '6'],
       ['3', '+', '2'],
       ['4', '+', '8'],
       ['6', '+', '4'],
       ['8', '+', '0'],
       ['9', '+', '6']],
      dtype='<U1')

In [52]:
chars = np.unique(xs)
chars

array(['+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype='<U1')

In [53]:
vocab_size = len(chars)
print('total chars:', vocab_size)

total chars: 11


We want to be able to easily go from char to its index in the vocabulary and back. Let's use dictionaries for that.

In [54]:
char2idx = {c: idx for idx, c in enumerate(chars)}
idx2char = {idx: c for idx, c in enumerate(chars)}

Now we need to convert our xs to an array with indices.

In [55]:
char2idx_ufunc = np.vectorize(lambda c: char2idx[c])

In [56]:
x_idxs = char2idx_ufunc(xs)

In [57]:
x_idxs[::16]

array([[ 1,  0,  1],
       [ 2,  0,  7],
       [ 4,  0,  3],
       [ 5,  0,  9],
       [ 7,  0,  5],
       [ 9,  0,  1],
       [10,  0,  7]])

We are ready to rock :)

In [58]:
# Both xs and y are ordered - this sort of selection of the val set will lead to the model never seeing '9' as x1
md = ColumnarModelData.from_arrays('.', [-10], x_idxs, y.astype(np.float32), bs=5)

Incremental model construction.

Will not go into my next post as it would make the code too messy. Wanted to share as I find it very useful.

In [59]:
class Calculator(nn.Module):
    def __init__(self, vocab_size, n_fac):
        super().__init__()
        
        # many simplifications here, which is good -
        #   I'll make the model only as complex as it needs to be to learn
        self.n_hidden = n_fac
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_hidden = nn.Linear(self.n_hidden, self.n_hidden)
        self.l_out = nn.Linear(self.n_hidden, 1)
    
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, self.n_hidden).cuda())
        
        for c in cs:
            inp = self.e(c)
            h = inp + self.l_hidden(h)
        
        return self.l_out(h)

In [60]:
m = Calculator(vocab_size, 1).cuda()

In [61]:
opt = optim.Adam(m.parameters(), 1)

fit(m, md, 4, opt, F.mse_loss)
set_lrs(opt, 1e-1)
fit(m, md, 6, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=4), HTML(value='')))

[  0.       32.04617  38.10055]                  
[  1.       28.68811   0.04945]                  
[  2.       21.28028   0.15449]                  
[  3.       14.71513   1.75965]                  



HBox(children=(IntProgress(value=0, description='Epoch', max=6), HTML(value='')))

[ 0.       1.95396  0.33035]                     
[ 1.       1.01763  0.33364]                     
[ 2.       0.57448  0.01587]                      
[ 3.       0.34325  0.0167 ]                      
[ 4.       0.21314  0.00184]                      
[ 5.       0.13564  0.00008]                      



The NN has very few parameters so it is quite susceptible to local minima. You might have to restart the training a couple of times to get nice results as above.

Now the fun thing! What has our NN learned?

In [62]:
chars

array(['+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype='<U1')

In [63]:
m.e.weight

Parameter containing:
 0.1344
 5.5938
 4.5990
 3.6036
 2.6170
 1.6118
 0.6123
-0.3791
-1.3487
-2.3770
-3.3730
[torch.cuda.FloatTensor of size 11x1 (GPU 0)]

In [64]:
m.l_hidden.weight

Parameter containing:
-1.0000
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]

In [65]:
m.l_out.weight

Parameter containing:
-1.0046
[torch.cuda.FloatTensor of size 1x1 (GPU 0)]

The network can converge to various solutions - with the hidden weight being a very small number close to zero, or one close to +1 or -1, etc. You might want to rerun the training and see what happens!

I tried adding a tanh activation between the hidden states but it didn't work. Makes sense if you think about what the network is doing. 

Can we get this to work with a hidden state and a tanh nonlinearity?

In [66]:
class CalculatorWithNonlinearities(nn.Module):
    def __init__(self, vocab_size, n_fac, n_hidden):
        super().__init__()

        self.n_hidden = n_hidden
        self.e = nn.Embedding(vocab_size, n_fac)
        self.l_in = nn.Linear(n_fac, n_hidden) # we need this because now n_fac != n_hidden
        self.l_hidden = nn.Linear(n_hidden, n_hidden)
        self.l_out = nn.Linear(n_hidden, 1)
    
    def forward(self, *cs):
        bs = cs[0].size(0)
        h = V(torch.zeros(bs, self.n_hidden).cuda())
        for c in cs:
            inp = self.l_in(self.e(c))
            h = F.tanh(self.l_hidden(h + inp))
        return self.l_out(h)

I know that 1 should work for the n_fac - the NN just needs to learn the number corresponding to each char. But what about n_hidden?

How small can I make the network for it to learn?

In [78]:
m = CalculatorWithNonlinearities(vocab_size, 1, 20).cuda()

In [79]:
opt = optim.Adam(m.parameters(), 1e-2)

fit(m, md, 30, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=30), HTML(value='')))

[  0.       63.47354  20.88388]                  
[  1.       40.15991   1.33555]                  
[  2.       28.93934   0.03963]                  
[  3.       24.02901   0.15532]                  
[  4.       20.99171   1.91515]                  
[  5.       18.52672   3.33561]                  
[  6.       16.63677   8.62258]                  
[  7.       14.37636  13.87331]                            
[  8.       12.12078   2.99008]                  
[ 9.       9.95543  2.44918]                     
[ 10.       7.7617   2.1268]                     
[ 11.        5.88463   0.85365]                  
[ 12.        4.41386   0.59487]                  
[ 13.        3.29314   0.21375]                  
[ 14.        2.44597   0.15518]                  
[ 15.        1.81645   0.26361]                  
[ 16.        1.34163   0.11318]                  
[ 17.        1.00977   0.08749]                  
[ 18.        0.76507   0.17916]                   
[ 19.        0.58762   0.14145]        

For what it's worth, 10 dimensionsional hidden state seems to be too small for this method of training / architecture. 20 seems to do the trick.

Let's see what embeddings it learned.

In [69]:
chars

array(['+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype='<U1')

In [80]:
m.e.weight

Parameter containing:
 0.2250
 1.5479
 1.0827
 0.7148
 0.3693
 0.0797
-0.1757
-0.3648
-0.6106
-0.8920
-1.2479
[torch.cuda.FloatTensor of size 11x1 (GPU 0)]

Makes sense! Now time for getting it to work with the nn.RNN black box.

Pasting the equation summarizing what it does here so that I know what to feed it and what I can expect to come out of it:
$$h_t = \tanh(w_{ih} * x_t + b_{ih}  +  w_{hh} * h_{(t-1)} + b_{hh})$$

In [71]:
class CalculatorWithRNNCell(nn.Module):
    def __init__(self, vocab_size, n_fac, n_hidden):
        super().__init__()

        self.n_hidden = n_hidden
        self.e = nn.Embedding(vocab_size, n_fac)   
        self.rnn = nn.RNN(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, 1)
    
    def forward(self, *cs):
        bs = cs[0].size(0)
        inp = self.e(torch.stack(cs))
        h0 = V(torch.zeros(1, bs, self.n_hidden).cuda())
        out, _ = self.rnn(inp, h0)
        
        return self.l_out(out[-1])

In [72]:
m = CalculatorWithRNNCell(vocab_size, 1, 20).cuda()

In [73]:
opt = optim.Adam(m.parameters(), 1e-2)

fit(m, md, 30, opt, F.mse_loss)

HBox(children=(IntProgress(value=0, description='Epoch', max=30), HTML(value='')))

[  0.       55.52471  12.62594]                            
[  1.       34.50159   0.06954]                            
[  2.       26.17814   0.1129 ]                  
[  3.       22.08547   0.00005]                  
[  4.       20.19321   0.00664]                            
[  5.       18.97991   0.08101]                            
[  6.       17.95803   0.1384 ]                            
[  7.       17.04634   0.18264]                          
[  8.       15.4479    1.29831]                            
[  9.       13.28404   0.43426]                            
[ 10.       10.60576   7.726  ]                            
[ 11.        8.26296  12.18458]                            
[ 12.        6.38482   7.41186]                            
[ 13.        4.99916   4.71922]                            
[ 14.        3.89386   5.31201]                            
[ 15.        3.0579    4.78369]                            
[ 16.        2.40439   3.62742]                  
[ 17.       

In [74]:
chars

array(['+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype='<U1')

In [75]:
m.e.weight

Parameter containing:
 1.2563
 2.5239
 1.6003
 1.0074
 0.6406
 0.3360
 0.0291
-0.3471
-0.8477
-1.5188
-2.1275
[torch.cuda.FloatTensor of size 11x1 (GPU 0)]

## End of part 1

And that's it for part 1! In part 2 we will look at longer sequences and specifically LTSM and GRU RNNs! 

See you in part 2!

### Fun things to try

* The RNNs for the learning from speeches part have not been tuned. Can you lower the validation loss by picking different embedding / hidden state dimensions? What about learning rate annealing (decreasing the learning rate as you train) or adding batch norm / dropout? 
* Can you teach the network to perform other calculations, such as subtraction or multiplication? How big of a hidden state do you need to make it work?
* What about a network that would take in long sequences of ones and zeros and count the ones? How long can we make such a sequence before the network is unable to learn?